# Part II – Machine Learning

> ## English Pipeline (Works for english only)

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers
from keras.utils import plot_model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return x1


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def remove_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text(text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = remove_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [ ]:
normalize_text("the baby is crying cried")

'baby cry cry'

> ## Arabic Pipeline (Works for arabic only)

In [ ]:
%pip install camel-tools

In [ ]:
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

!export | camel_data full

In [ ]:
# For removing punctuation
import string
# Some characters have different variants, for example the character 'ع' can also be represented 
# as 'ﻊ', 'ﻋ', 'ﻌ' depending on where it appears in a word. For all these forms, we would call 'ع' their canonical form.
from camel_tools.utils.normalize import normalize_unicode


# It is common for Arabic speakers to use shortcuts when typing Arabic text. 
# For example, the different variants of the letter alef ('ا', 'آ', 'أ', 'إ') may be typed as just 'ا'.
# هل ذهبت إلى المكتبة؟
# هل ذهبت الى المكتبة؟
# هل ذهبت الي المكتبة؟
# هل ذهبت الي المكتبه؟
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

# To split by whitespace and seperate punctuation, we currently provide the utility function
# هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
# ['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ', '؟']
from camel_tools.tokenizers.word import simple_word_tokenize


# هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
# هل ذهبت إلى المكتبة؟
# To remove arabic diacritical marks
# Either use this strategy or the strategy that involves adding the most likely
# diacritical marks coming 
#### was not used
from camel_tools.utils.dediac import dediac_ar


# Load a pretrained disambiguator to use with a tokenizer
from camel_tools.disambig.mle import MLEDisambiguator

# we can specify parameter to separate "فذهب" to 
# ["ذهب",
#     "+ف"]
# we can also specify parameters to add diacritical marks based on a maximum likelihod estimation algorithm
from camel_tools.tokenizers.morphological import MorphologicalTokenizer




# Removing punctuation from text
# string.punctuation = !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
def remove_punctuation(text):
  text = text.translate(str.maketrans('', '', string.punctuation))
  return text


# Remove english (not best technique there could be better)
def remove_eng(text):
   return re.sub(r'[A-Za-z]', '', text)


# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# Add them all into one pipeline
def normalize_text(text):
  text = remove_punctuation(text)
  text = remove_eng(text)
  text = normalize_unicode(text)
  text = normalize_alef_maksura_ar(text)
  text = normalize_alef_ar(text)
  text = normalize_teh_marbuta_ar(text)

  text = simple_word_tokenize(text)

  # We can output diacritized tokens by setting `diac=True`
  tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True, diac=True)
  return ' '.join(tokenizer.tokenize(text))

In [ ]:
arabic_text = 'لوريم ايبسوم هو نموذج افتراضي يوضع في التصاميم لتعرض على العميل ليتصور طريقه وضع النصوص بالتصاميم سواء كانت تصاميم مطبوعه ... بروشور او فلاير على سبيل المثال ... او نماذج www.justawebsite.com مواقع انترنت ... وعند موافقه العميل المبدئيه على التصميم يتم ازالة هذا النص من التصميم ويتم وضع النصوص النهائية المطلوبة للتصميم و يقول البعض ان وضع النصوص التجريبية بالتصميم قد تشغل justanemail@gmail.com المشاهد عن وضع الكثير من الملاحظات او الانتقادات للتصميم الاساسي . وخلافا للاعتقاد السائد فإن لوريم إيبسوم ليس نصا عشوائيا،ً بل إن له جذور في الأدب اللاتيني الكلاسيكي منذ العام 45 قبل الميلاد. من كتاب " حول أقاصي الخير والشر "'
arabic_text.split()

['لوريم',
 'ايبسوم',
 'هو',
 'نموذج',
 'افتراضي',
 'يوضع',
 'في',
 'التصاميم',
 'لتعرض',
 'على',
 'العميل',
 'ليتصور',
 'طريقه',
 'وضع',
 'النصوص',
 'بالتصاميم',
 'سواء',
 'كانت',
 'تصاميم',
 'مطبوعه',
 '...',
 'بروشور',
 'او',
 'فلاير',
 'على',
 'سبيل',
 'المثال',
 '...',
 'او',
 'نماذج',
 'www.justawebsite.com',
 'مواقع',
 'انترنت',
 '...',
 'وعند',
 'موافقه',
 'العميل',
 'المبدئيه',
 'على',
 'التصميم',
 'يتم',
 'ازالة',
 'هذا',
 'النص',
 'من',
 'التصميم',
 'ويتم',
 'وضع',
 'النصوص',
 'النهائية',
 'المطلوبة',
 'للتصميم',
 'و',
 'يقول',
 'البعض',
 'ان',
 'وضع',
 'النصوص',
 'التجريبية',
 'بالتصميم',
 'قد',
 'تشغل',
 'justanemail@gmail.com',
 'المشاهد',
 'عن',
 'وضع',
 'الكثير',
 'من',
 'الملاحظات',
 'او',
 'الانتقادات',
 'للتصميم',
 'الاساسي',
 '.',
 'وخلافا',
 'للاعتقاد',
 'السائد',
 'فإن',
 'لوريم',
 'إيبسوم',
 'ليس',
 'نصا',
 'عشوائيا،ً',
 'بل',
 'إن',
 'له',
 'جذور',
 'في',
 'الأدب',
 'اللاتيني',
 'الكلاسيكي',
 'منذ',
 'العام',
 '45',
 'قبل',
 'الميلاد.',
 'من',
 'كتاب',
 '"',
 'حو

In [ ]:
normalized_arabic_text = normalize_text(arabic_text)
print(normalized_arabic_text)

لوريم ايبسوم هُوَ نَمُوذَجَ اِفْتِراضِيٌّ يُوضَع فِي ال+ تَصامِيم لَ+ تَعَرَّضَ عَلَى ال+ عَمِيلِ لِ+ يَتَصَوَّر طَرِيقِ +هُ وَضْعِ ال+ نُصُوصِ بِ+ ال+ تَصامِيمِ سَواءُ كانَت تَصامِيم مَطْبُوعَة بروشور أَوْ فَ+ لِ+ إِير عَلَى سَبِيلِ ال+ مِثالِ أَوْ نَماذِجَ مَواقِعِ إِنْتِرْنِت وَ+ عِنْد مُوافَقَة ال+ عَمِيلِ ال+ مَبْدَئِيَّة عَلَى ال+ تَصْمِيمِ يَتِمّ أَزال +هُ هٰذا ال+ نَصِّ مِن ال+ تَصْمِيمِ وَ+ يَتِمّ وَضْعِ ال+ نُصُوصِ ال+ نِهائِيَّة ال+ مَطْلُوبَة لِ+ ال+ تَصْمِيم وَ يَقُول ال+ بَعْضُ أَنَّ وَضْعِ ال+ نُصُوصِ ال+ تَجْرِيبِيَّة بِ+ ال+ تَصْمِيم قَدْ تَشْغَل ال+ مَشاهِدِ عَن وَضْعِ ال+ كَثِيرَ مِن ال+ مُلاحَظاتِ أَوْ ال+ اِنْتِقاداتِ لِ+ ال+ تَصْمِيم ال+ أَساسِيِّ وَ+ خِلافاً لِ+ ال+ اِعْتِقاد ال+ سائِدِ فَ+ إِنَّ لوريم ايبسوم لَيِسَ نَصّاً عَشْوائِيّاً ، ً بَلْ أَنَّ لِ +هُ جُذُورُ فِي ال+ أَدَبِ ال+ لاتِينِيّ ال+ كلاسِيكِيِّ مُنْذُ ال+ عامِّ 45 قِبَلَ ال+ مِيلادِ مِن كِتابِ حَوْلَ أَقاصِي ال+ خَيْرِ وَ+ ال+ شَرِّ


In [ ]:
# Length of the output list
len(normalized_arabic_text.split())

148

> ## Tokenization

In [ ]:
# Creating Our Tokenizer
vocab_size = 50 #we can let it be 50 for now as we only have one example
num_words=vocab_size


tok = Tokenizer(num_words = vocab_size , oov_token='UNK')
tok.fit_on_texts([normalized_arabic_text])

In [ ]:
# Inspecting word index
tok.word_index

{'45': 76,
 'UNK': 1,
 '،': 67,
 'أَدَبِ': 71,
 'أَزال': 44,
 'أَساسِيِّ': 59,
 'أَقاصِي': 81,
 'أَنَّ': 21,
 'أَوْ': 9,
 'إِنَّ': 63,
 'إِنْتِرْنِت': 40,
 'إِير': 35,
 'ال': 2,
 'ايبسوم': 13,
 'اِعْتِقاد': 61,
 'اِفْتِراضِيٌّ': 24,
 'اِنْتِقاداتِ': 58,
 'بروشور': 34,
 'بَعْضُ': 50,
 'بَلْ': 69,
 'بِ': 17,
 'تَجْرِيبِيَّة': 51,
 'تَشْغَل': 53,
 'تَصامِيم': 15,
 'تَصامِيمِ': 30,
 'تَصْمِيم': 11,
 'تَصْمِيمِ': 19,
 'تَعَرَّضَ': 27,
 'جُذُورُ': 70,
 'حَوْلَ': 80,
 'خَيْرِ': 82,
 'خِلافاً': 60,
 'سائِدِ': 62,
 'سَبِيلِ': 36,
 'سَواءُ': 31,
 'شَرِّ': 83,
 'طَرِيقِ': 29,
 'عامِّ': 75,
 'عَشْوائِيّاً': 66,
 'عَلَى': 6,
 'عَمِيلِ': 16,
 'عَن': 55,
 'عِنْد': 41,
 'فَ': 18,
 'فِي': 14,
 'قَدْ': 52,
 'قِبَلَ': 77,
 'كانَت': 32,
 'كلاسِيكِيِّ': 73,
 'كَثِيرَ': 56,
 'كِتابِ': 79,
 'لاتِينِيّ': 72,
 'لوريم': 12,
 'لَ': 26,
 'لَيِسَ': 64,
 'لِ': 3,
 'مَبْدَئِيَّة': 43,
 'مَشاهِدِ': 54,
 'مَطْبُوعَة': 33,
 'مَطْلُوبَة': 48,
 'مَواقِعِ': 39,
 'مُلاحَظاتِ': 57,
 'مُنْذُ': 74,
 'مُوافَقَة': 42,
 'مِثالِ'

In [ ]:
# This is the document with its words index in the document
tok.texts_to_sequences([normalized_arabic_text.split()])

[[12,
  13,
  22,
  23,
  24,
  25,
  14,
  1,
  15,
  1,
  27,
  6,
  1,
  16,
  1,
  28,
  29,
  1,
  5,
  1,
  8,
  1,
  1,
  30,
  31,
  32,
  15,
  33,
  34,
  9,
  1,
  1,
  35,
  6,
  36,
  1,
  37,
  9,
  38,
  39,
  40,
  1,
  41,
  42,
  1,
  16,
  1,
  43,
  6,
  1,
  19,
  20,
  44,
  1,
  45,
  1,
  46,
  10,
  1,
  19,
  1,
  20,
  5,
  1,
  8,
  1,
  47,
  1,
  48,
  1,
  1,
  11,
  4,
  49,
  1,
  1,
  21,
  5,
  1,
  8,
  1,
  1,
  1,
  1,
  11,
  1,
  1,
  1,
  1,
  1,
  5,
  1,
  1,
  10,
  1,
  1,
  9,
  1,
  1,
  1,
  1,
  11,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  12,
  13,
  1,
  1,
  1,
  1,
  1,
  1,
  21,
  3,
  1,
  1,
  14,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  10,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]]

In [ ]:
# This the representation in binary
# it's all 1 because we only have one example
tok.texts_to_matrix([normalized_arabic_text], mode='binary')

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]])

__The text is now ready to be fed in a neural network__

# Citation and references

In [ ]:
# https://colab.research.google.com/drive/1Y3qCbD6Gw1KEw-lixQx1rI6WlyWnrnDS?usp=sharing#scrollTo=Upf_eeVE0889
# https://wti.kaust.edu.sa/upcoming-events/Machine-Learning-Arabic-NLP-Webinar/ml-webinar-2020-keynote-1
# University of New York Abudhabi

# @inproceedings{obeid-etal-2020-camel,
#    title = "{CAM}e{L} Tools: An Open Source Python Toolkit for {A}rabic Natural Language Processing",
#    author = "Obeid, Ossama  and
#       Zalmout, Nasser  and
#       Khalifa, Salam  and
#       Taji, Dima  and
#       Oudah, Mai  and
#       Alhafni, Bashar  and
#       Inoue, Go  and
#       Eryani, Fadhl  and
#       Erdmann, Alexander  and
#       Habash, Nizar",
#    booktitle = "Proceedings of the 12th Language Resources and Evaluation Conference",
#    month = may,
#    year = "2020",
#    address = "Marseille, France",
#    publisher = "European Language Resources Association",
#    url = "https://www.aclweb.org/anthology/2020.lrec-1.868",
#    pages = "7022--7032",
#    abstract = "We present CAMeL Tools, a collection of open-source tools for Arabic natural language processing in Python. CAMeL Tools currently provides utilities for pre-processing, morphological modeling, Dialect Identification, Named Entity Recognition and Sentiment Analysis. In this paper, we describe the design of CAMeL Tools and the functionalities it provides.",
#    language = "English",
#    ISBN = "979-10-95546-34-4",
# }